In [ ]:
%matplotlib inline

import seaborn as sns
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import random

import sys
sys.path.append('../scripts/')

from training_electromagnetic import Trainer
from presets import LambdaSearch
from config_plots import global_settings
from utils import *
from parameters import Params

global_settings()

from fastprogress.fastprogress import master_bar, progress_bar

torch.cuda.set_device(0)
device = torch.device('cuda:0')

## Free Log Files

In [ ]:
num_logs = len(
    [
        name for name in os.listdir(
            '../logs/'
        ) if os.path.isfile(name)
    ]
)

if num_logs > 2000:
    free_logs('../logs/')

## Set Up Parameters and Start Tasks

This part includes customized training parameters. Change these to whatever you need.

In [ ]:
param_presets = LambdaSearch(data_path='//raid/elhamod/Eignedecomposition/Electromagnetic/electromagnetic_dualphase')

train_sizes = ['2000examples_400x400complex.mat']

datasets = []
break_loop = True
loss_plot = True

num_of_experiments = 1

mb = master_bar(range(num_of_experiments))
for i in mb:
    for train_size in progress_bar(train_sizes):
        
        param = param_presets.NSE_DNNex_overlap()
        param.data_params['train_size']=train_size
        param.name = 'CoPhy_PGNN'
        param.train_params['break_loop_early'] = break_loop
        param.train_params['num_batch'] = 10
        param.train_params['steplr'] = 80
        param.train_params['gamma'] = 0.99
        param.loss_params['lambda_e0'] = 1# Initial value of S_coeff
        param.loss_params['lambda_s'] = 0.0# Initial value of C_coeff
        param.loss_params['anneal_factor'] = 0.95# A lower value will decay S_coeff faster.
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.loss_params['cold_start'] = {
            'mode': 'sigmoid',
            'lambda_s': 0.0001,# Final value of C_coeff
            'threshold': 2000, #350
            'smooth': 0.01,
        }   
        param.train_params['epochs'] = 4000
        param.train_params['early_stopping'] = {'patience':4000, 'verbose':False, 'delta':0}
        param.nn_params['hidden_size'] = 100
        param.nn_params['depth'] = 2
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

<!-- 

# the black-box neural networks
param = param_presets.DNN()
param.data_params['train_size']=['2000examples_400x400complex.mat']
param.name = 'BB'
param.train_params['break_loop_early'] = break_loop
param.train_params['num_batch'] = 10
param.data_params['device'] = device
param.nn_params['device'] = device
param.train_params['epochs'] = 4000
param.train_params['early_stopping'] = {'patience':4000, 'verbose':False, 'delta':0}
param.nn_params['hidden_size'] = 100
param.nn_params['depth'] = 2
trainer = Trainer(master_bar=mb, plot=loss_plot)
trainer.start(param)

# CoPhy-PGNN
param = param_presets.NSE_DNNex()
param.data_params['train_size']=['2000examples_400x400complex.mat']
param.name = 'CoPhy'
param.train_params['break_loop_early'] = break_loop
param.train_params['num_batch'] = 10
param.loss_params['lambda_e0'] = 0.01# Initial value of S_coeff
param.loss_params['anneal_factor'] = 0.9# A lower value will decay S_coeff faster.
param.data_params['device'] = device
param.nn_params['device'] = device
param.loss_params['cold_start'] = {
    'mode': 'sigmoid',
    'lambda_s': 0.00000006,# Final value of C_coeff
    'threshold': 1500,
    'smooth': 0.003
}   
param.train_params['epochs'] = 4000
param.train_params['early_stopping'] = {'patience':4000, 'verbose':False, 'delta':0}
param.nn_params['hidden_size'] = 100
param.nn_params['depth'] = 2
trainer = Trainer(master_bar=mb, plot=loss_plot)
trainer.start(param) -->